In [240]:
import pandas as pd

In [241]:
data = pd.read_csv('merged_dataset.csv')
data = data.sort_values(by='Date').reset_index(drop=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_17152\1771675324.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('merged_dataset.csv')


In [242]:
# 'match_strength'

# 'League': Champions-League AND 'Round': 'Group Stage' OR 'League Phase' OR 'Round of 16' set strength to 50
# 'League': Champions-League AND 'Round': 'Quarter-finals' OR 'Semi-finals' OR 'Final' set strength to 60
# 'League': Champions-League AND 'Round': ELSE set strength to 25
# 'League': Europa-League AND 'Round': 'Group Stage' OR 'League Phase' OR 'Round of 16' set strength to 35
# 'League': Europa-League AND 'Round': 'Quarter-finals' OR 'Semi-finals' OR 'Final' set strength to 45
# 'League': Europa-League AND 'Round': ELSE set strength to 20
# 'League': Conference-League AND 'Round': 'Group Stage' OR 'League Phase' OR 'Round of 16' set strength to 20
# 'League': Conference-League AND 'Round': 'Quarter-finals' OR 'Semi-finals' OR 'Final' set strength to 30
# 'League': Conference-League AND 'Round': ELSE set strength to 15
# 'League': NOT IN ['Champions-League', 'Europa-League', 'Conference-League'] set strength to 10

def set_match_strength(row):
    if row['League'] == 'Champions-League':
        if row['Round'] in ['Group Stage', 'League Phase', 'Round of 16']:
            return 50
        elif row['Round'] in ['Quarter-finals', 'Semi-finals', 'Final']:
            return 60
        else:
            return 25
    elif row['League'] == 'Europa-League':
        if row['Round'] in ['Group Stage', 'League Phase', 'Round of 16']:
            return 35
        elif row['Round'] in ['Quarter-finals', 'Semi-finals', 'Final']:
            return 40
        else:
            return 20
    elif row['League'] == 'Conference-League':
        if row['Round'] in ['Group Stage', 'League Phase', 'Round of 16']:
            return 15
        elif row['Round'] in ['Quarter-finals', 'Semi-finals', 'Final']:
            return 20
        else:
            return 15
    else:
        return 10

In [243]:
data['match_strength'] = data.apply(set_match_strength, axis=1)

In [244]:
data[data['League']=='Champions-League']

,Season,Wk,Date,Home,Away,Home_Goals,Away_Goals,Result,League,Round,Home_Country,Away_Country,match_strength
39464,1990-1991,NaN,1990-09-18,Odense,Real Madrid,1.0,4.0,0.0,Champions-League,First round,dk,es,25
39466,1990-1991,NaN,1990-09-19,Knattspyrnufélag Akureyrar,CSKA Sofia,1.0,0.0,1.0,Champions-League,First round,is,bg,25
39474,1990-1991,NaN,1990-09-19,Marseille,KS Dinamo Tirana,5.0,1.0,1.0,Champions-League,First round,fr,al,25
39478,1990-1991,NaN,1990-09-19,Dinamo București,St Patrick's Athletic FC,4.0,0.0,1.0,Champions-League,First round,ro,ie,25
39479,1990-1991,NaN,1990-09-19,Valletta FC,Rangers,0.0,4.0,0.0,Champions-League,First round,mt,sct,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174848,2024-2025,NaN,2025-03-11,Leverkusen,Bayern Munich,NaN,NaN,0.5,Champions-League,Round of 16,de,de,50
174850,2024-2025,NaN,2025-03-12,Aston Villa,Club Brugge,NaN,NaN,0.5,Champions-League,Round of 16,eng,be,50
174852,2024-2025,NaN,2025-03-12,Lille,Dortmund,NaN,NaN,0.5,Champions-League,Round of 16,fr,de,50
174853,2024-2025,NaN,2025-03-12,Arsenal,PSV Eindhoven,NaN,NaN,0.5,Champions-League,Round of 16,eng,nl,50


In [245]:
data = data.sort_values(by='Date').reset_index(drop=True)

elo_ratings = {}

def after_elo(row):

    home = row['Home']
    away = row['Away']

    home_before = elo_ratings.get(home, 1500)
    away_before = elo_ratings.get(away, 1500)
    i = row['match_strength']

    we_home = 1 / ( ( 10 ** ( - ( home_before - away_before ) / 600 ) ) + 1 )
    we_away = 1 - we_home

    home_after = home_before + ( ( row['Result'] - we_home ) * i )
    away_after = away_before + ( ( ( 1 - row['Result'] ) - we_away ) * i )

    elo_ratings[home] = home_after
    elo_ratings[away] = away_after

    return home_after, away_after

In [246]:
data

,Season,Wk,Date,Home,Away,Home_Goals,Away_Goals,Result,League,Round,Home_Country,Away_Country,match_strength
0,1888-1889,1.0,1888-09-08,Preston,Burnley,5.0,2.0,1.0,Premier-League,NaN,NaN,NaN,10
1,1888-1889,1.0,1888-09-08,Everton,Accrington,2.0,1.0,1.0,Premier-League,NaN,NaN,NaN,10
2,1888-1889,1.0,1888-09-08,Wolves,Aston Villa,1.0,1.0,0.5,Premier-League,NaN,NaN,NaN,10
3,1888-1889,1.0,1888-09-08,Bolton,Derby County,3.0,6.0,0.0,Premier-League,NaN,NaN,NaN,10
4,1888-1889,1.0,1888-09-08,Stoke,West Brom,0.0,2.0,0.0,Premier-League,NaN,NaN,NaN,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176766,2025,30.0,2025-11-30,Sandefjord,KFUM Oslo,NaN,NaN,NaN,Eliteserien,NaN,NaN,NaN,10
176767,2025,30.0,2025-11-30,Sarpsborg 08,Molde,NaN,NaN,NaN,Eliteserien,NaN,NaN,NaN,10
176768,2025,30.0,2025-11-30,Viking,Vålerenga,NaN,NaN,NaN,Eliteserien,NaN,NaN,NaN,10
176769,2025,30.0,2025-11-30,Rosenborg,Strømsgodset,NaN,NaN,NaN,Eliteserien,NaN,NaN,NaN,10


In [247]:
# data[['Elo_Home', 'Elo_Away']] = data.apply(after_elo, axis=1, result_type='expand')

In [248]:
data = data.dropna(subset=['Result'])
data

,Season,Wk,Date,Home,Away,Home_Goals,Away_Goals,Result,League,Round,Home_Country,Away_Country,match_strength
0,1888-1889,1.0,1888-09-08,Preston,Burnley,5.0,2.0,1.0,Premier-League,NaN,NaN,NaN,10
1,1888-1889,1.0,1888-09-08,Everton,Accrington,2.0,1.0,1.0,Premier-League,NaN,NaN,NaN,10
2,1888-1889,1.0,1888-09-08,Wolves,Aston Villa,1.0,1.0,0.5,Premier-League,NaN,NaN,NaN,10
3,1888-1889,1.0,1888-09-08,Bolton,Derby County,3.0,6.0,0.0,Premier-League,NaN,NaN,NaN,10
4,1888-1889,1.0,1888-09-08,Stoke,West Brom,0.0,2.0,0.0,Premier-League,NaN,NaN,NaN,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176349,2024-2025,38.0,2025-05-25,Girona,Atlético Madrid,NaN,NaN,0.5,La-Liga,NaN,NaN,NaN,10
176352,2024-2025,38.0,2025-05-25,Tottenham,Brighton,NaN,NaN,0.5,Premier-League,NaN,NaN,NaN,10
176353,2024-2025,38.0,2025-05-25,Wolves,Brentford,NaN,NaN,0.5,Premier-League,NaN,NaN,NaN,10
176354,2024-2025,38.0,2025-05-25,Manchester Utd,Aston Villa,NaN,NaN,0.5,Premier-League,NaN,NaN,NaN,10


In [249]:
data.isna().sum()

Season                 0
Wk                  5537
Date                   0
Home                   0
Away                   0
Home_Goals          1175
Away_Goals          1175
Result                 0
League                 0
Round             128350
Home_Country      164195
Away_Country      164195
match_strength         0
dtype: int64

In [250]:
for index, row in data.iterrows():
    after_elo(row)

In [251]:
ranking = sorted(elo_ratings.items(), key=lambda x: x[1], reverse=True)

# Print ranking
for rank, (team, elo) in enumerate(ranking, start=1):
    print(f"{rank}. {team}: {elo:.2f}")

1. Real Madrid: 2070.50
2. Manchester City: 2014.88
3. Liverpool: 1989.50
4. Bayern Munich: 1989.38
5. Inter: 1969.33
6. Barcelona: 1943.77
7. Arsenal: 1932.72
8. Atlético Madrid: 1925.22
9. Paris S-G: 1911.05
10. Leverkusen: 1904.99
11. PSV Eindhoven: 1876.02
12. Benfica: 1872.97
13. Dortmund: 1863.44
14. Red Star: 1856.82
15. Feyenoord: 1848.82
16. Atalanta: 1846.49
17. Fenerbahçe: 1834.96
18. Galatasaray: 1833.49
19. Napoli: 1830.07
20. Chelsea: 1826.01
21. Juventus: 1825.83
22. Sporting CP: 1820.79
23. Celtic: 1820.24
24. Porto: 1817.28
25. Milan: 1804.53
26. Olympiacos: 1800.78
27. Lazio: 1799.71
28. Slavia Prague: 1797.96
29. Rangers: 1795.54
30. Roma: 1794.12
31. Club Brugge: 1794.06
32. Partizan: 1788.89
33. Ajax: 1788.56
34. Villarreal: 1781.14
35. Manchester Utd: 1781.06
36. Dinamo Zagreb: 1779.79
37. Shakhtar: 1779.75
38. RB Leipzig: 1779.55
39. Ludogorets: 1779.51
40. Lille: 1776.30
41. Athletic Club: 1772.92
42. Viktoria Plzeň: 1761.73
43. Zenit: 1755.55
44. Real Sociedad: